In [1]:
%load_ext autoreload
%autoreload 2

import os
import torch
import numpy as np
from glob import glob
from tqdm.auto import tqdm
from sklearn.linear_model import LogisticRegression
import pandas as pd
from utils.store_data import ReadData
import numpy as np
import torch
from utils.arguments import Arguments
from tqdm import tqdm
from sklearn.decomposition import PCA
from linear_probe import get_linear_acc
import random
import os
from sklearn.metrics import roc_auc_score,accuracy_score,roc_curve
#from answer_judge.bleurt import BleurtJudge
from answer_judge.vqaEval import VQAEval

In [2]:
data_reader=ReadData('/root/autodl-tmp/hallscope/VQA/train')
data=data_reader.read_all()
random.shuffle(data)

In [3]:
df=pd.DataFrame(data)

In [4]:
df.head()

,img_path,question,answers,question_id,most_likely
0,[/root/autodl-fs/coco_images/train/COCO_train2...,[Are these computer connected through a networ...,"[{'answer': ['no', 'table'], 'answer_confidenc...","[tensor(366527002), tensor(392319002)]",{'embedding': [[[ 0.00389099 0.00149536 0.00...
1,[/root/autodl-fs/coco_images/train/COCO_train2...,[Is the hydrant all one color?\nAnswer the que...,"[{'answer': ['yes', 'hippo and elephant'], 'an...","[tensor(392315001), tensor(328651001)]",{'embedding': [[[ 0.01013184 0.00717163 -0.00...
2,[/root/autodl-fs/coco_images/train/COCO_train2...,[What type of train are the people sitting on?...,"[{'answer': ['commuter', 'no'], 'answer_confid...","[tensor(286051000), tensor(183435006)]",{'embedding': [[[ 0.0032196 0.00233459 0.01...
3,[/root/autodl-fs/coco_images/train/COCO_train2...,[Which room is depicted in the picture?\nAnswe...,"[{'answer': ['bathroom', 'green'], 'answer_con...","[tensor(265324001), tensor(535049002)]",{'embedding': [[[ 0.01403809 -0.00366211 0.03...
4,[/root/autodl-fs/coco_images/train/COCO_train2...,[What animal is pictured?\nAnswer the question...,"[{'answer': ['goat', 'bat'], 'answer_confidenc...","[tensor(498769001), tensor(402081002)]",{'embedding': [[[-0.01745605 0.01733398 -0.00...


In [5]:
df.shape

(5000, 5)

In [6]:
i=0

In [7]:
df.iloc[i]['img_path']

['/root/autodl-fs/coco_images/train/COCO_train2014_000000366527.jpg',
 '/root/autodl-fs/coco_images/train/COCO_train2014_000000392319.jpg']

In [8]:
df.iloc[i]['most_likely']['response']

['Yes', 'Menu']

In [9]:
df.iloc[i]['question']

['Are these computer connected through a network?\nAnswer the question using a single word or phrase.\n',
 'What is under the paper?\nAnswer the question using a single word or phrase.\n']

In [10]:
df.iloc[i]['answers']

[{'answer': ['no', 'table'],
  'answer_confidence': ['no', 'yes'],
  'answer_id': tensor([1, 1])},
 {'answer': ['yes', 'menu'],
  'answer_confidence': ['yes', 'maybe'],
  'answer_id': tensor([2, 2])},
 {'answer': ['yes', 'table'],
  'answer_confidence': ['yes', 'maybe'],
  'answer_id': tensor([3, 3])},
 {'answer': ['yes', 'table'],
  'answer_confidence': ['yes', 'yes'],
  'answer_id': tensor([4, 4])},
 {'answer': ['yes', 'menu'],
  'answer_confidence': ['maybe', 'maybe'],
  'answer_id': tensor([5, 5])},
 {'answer': ['yes', 'table'],
  'answer_confidence': ['maybe', 'yes'],
  'answer_id': tensor([6, 6])},
 {'answer': ['yes', 'menu'],
  'answer_confidence': ['no', 'maybe'],
  'answer_id': tensor([7, 7])},
 {'answer': ['no', 'table'],
  'answer_confidence': ['maybe', 'maybe'],
  'answer_id': tensor([8, 8])},
 {'answer': ['no', 'menu'],
  'answer_confidence': ['maybe', 'maybe'],
  'answer_id': tensor([9, 9])},
 {'answer': ['probably', 'beer'],
  'answer_confidence': ['maybe', 'yes'],
  'an

In [11]:
embeddings=df['most_likely'].apply(lambda x: x['embedding']).to_list()

In [12]:
embeddings=np.concatenate(embeddings,axis=0)

In [13]:
embeddings.shape

(10000, 33, 4096)

In [14]:
def transpose_lines(x):
    f=pd.DataFrame(x)
    transposed=f['answer'].apply(pd.Series).T.values.tolist()
    return transposed

df['answers']=df['answers'].apply(transpose_lines)

In [15]:
gt_label=df['answers'].to_list()
gt_label=np.concatenate(gt_label,axis=0).tolist()

In [16]:
responses=df['most_likely'].apply(lambda x: x['response']).to_list()
rsps=[]
for x in responses:
    rsps+=x

In [17]:
rsps[0]

'Yes'

In [18]:
judge=VQAEval(gt_label,rsps)
judge.evaluate()
print(judge.evalQA)
print(judge.accuracy['overall'])


computing accuracy


  3%|▎         | 286/10000 [00:00<00:03, 2856.43it/s]

100%|██████████| 10000/10000 [00:03<00:00, 2860.58it/s]

Done computing accuracy
{0: 100, 1: 100, 2: 0.0, 3: 100, 4: 66.67, 5: 100, 6: 100, 7: 100, 8: 100, 9: 100, 10: 100, 11: 0.0, 12: 100, 13: 66.67, 14: 100, 15: 100, 16: 33.33, 17: 33.33, 18: 100, 19: 100, 20: 0.0, 21: 0.0, 22: 0.0, 23: 100, 24: 100, 25: 100, 26: 100, 27: 100, 28: 0.0, 29: 100, 30: 100, 31: 66.67, 32: 0.0, 33: 0.0, 34: 100, 35: 0.0, 36: 100, 37: 100, 38: 100, 39: 100, 40: 100, 41: 0.0, 42: 66.67, 43: 100, 44: 0.0, 45: 33.33, 46: 33.33, 47: 100, 48: 100, 49: 100, 50: 0.0, 51: 100, 52: 100, 53: 0.0, 54: 100, 55: 100, 56: 100, 57: 100, 58: 100, 59: 100, 60: 100, 61: 100, 62: 100, 63: 66.67, 64: 100, 65: 100, 66: 100, 67: 100, 68: 100, 69: 100, 70: 100, 71: 100, 72: 100, 73: 100, 74: 100, 75: 100, 76: 100, 77: 0.0, 78: 100, 79: 0.0, 80: 0.0, 81: 100, 82: 100, 83: 100, 84: 100, 85: 33.33, 86: 100, 87: 100, 88: 100, 89: 100, 90: 100, 91: 100, 92: 100, 93: 100, 94: 100, 95: 100, 96: 66.67, 97: 0.0, 98: 100, 99: 100, 100: 100, 101: 100, 102: 100, 103: 66.67, 104: 100, 105: 33.33,

In [19]:
gt_scores=[judge.evalQA[i] for i in range(len(judge.evalQA))]

In [20]:
is_correct=[]
threshold=10
for i,x in enumerate(gt_scores):
    if x >=threshold:
        is_correct.append(1)
    else:
        is_correct.append(0)
is_correct=np.array(is_correct)

In [21]:
is_correct.shape

(10000,)

In [22]:
print(f'There are {len(is_correct)} responses. {sum(is_correct==1)} answer correctly and {sum(is_correct==0)} answer wrongly')

There are 10000 responses. 8570 answer correctly and 1430 answer wrongly


In [23]:
length=len(is_correct)
train_rate=0.8

all_indices = np.random.permutation(length)
train_val_idxs = all_indices[:int(
    train_rate * length)]  # trainset and validation index

test_idxs=all_indices[int(
    train_rate * length):]  #test index

validset_len = 1000
# exclude validation samples.
train_idxs = train_val_idxs[:len(
    train_val_idxs) - validset_len]  # trainset index
val_idxs = train_val_idxs[len(
    train_val_idxs) - validset_len:]  # validation index
hall_label_test = []
hall_label_wild = []
hall_label_val = []

'''get testset, wildset and valset. The valset is used for determining the hype-parameters'''
hall_label_test = is_correct[test_idxs]
hall_label_wild = is_correct[train_idxs]
hall_label_val = is_correct[val_idxs]

print(f'trainset length:{len(hall_label_wild)}')
print(f'validset length:{len(hall_label_val)}')
print(f'testset length:{len(hall_label_test)}')
print(f'trainset postive: {sum(hall_label_wild==1)} negative: {sum(hall_label_wild==0)}')
print(f'valset postive: {sum(hall_label_val==1)} negative: {sum(hall_label_val==0)}')
print(
    f'testset postive: {sum(hall_label_test==1)} negative: {sum(hall_label_test==0)}')

trainset length:7000
validset length:1000
testset length:2000
trainset postive: 6032 negative: 968
valset postive: 850 negative: 150
testset postive: 1688 negative: 312


In [24]:
all_embeddings = embeddings
all_embeddings.shape

(10000, 33, 4096)

In [25]:
embedding_train = all_embeddings[train_idxs]
embedding_val = all_embeddings[val_idxs]
embedding_test = all_embeddings[test_idxs]

In [26]:
%load_ext autoreload
%autoreload 2
from ml_utils.grid_search import GridSearch
from ml_utils.PCA_discriminator import PCADiscriminator
from ml_utils.metrics import auroc,get_best_split_from_scores

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Grid Search for Best 'n_components' & 'layer'

In [27]:
def evaluator(n_components,i_layer,X,y):
    discriminator=PCADiscriminator(n_components,X[:,i_layer,:])
    scores=discriminator.get_score()
    return auroc(scores,y)

In [28]:
# graid search for best hyper-parameters on validation set
grid={
    'n_components':range(1,12),
    'layer':range(embedding_val.shape[1])
}
grid_search=GridSearch(evaluator,grid,embedding_val,hall_label_val)
best_paras=grid_search.search()

Grid Searching for best n_components,layer


  0%|          | 0/11 [00:00<?, ?it/s]

found best n_components:1,layer:0 Score: 0.6672156862745098
found best n_components:1,layer:2 Score: 0.6986823529411764
found best n_components:1,layer:16 Score: 0.7089411764705882


  9%|▉         | 1/11 [00:01<00:19,  1.93s/it]

found best n_components:2,layer:16 Score: 0.7123372549019608
found best n_components:2,layer:17 Score: 0.7324470588235295
found best n_components:2,layer:19 Score: 0.7355686274509803
found best n_components:2,layer:20 Score: 0.7390588235294118


 18%|█▊        | 2/11 [00:03<00:16,  1.82s/it]

found best n_components:3,layer:17 Score: 0.7457098039215686


100%|██████████| 11/11 [00:21<00:00,  1.96s/it]


In [29]:
best_paras

{'n_components': 3, 'layer': 17, 'best_score': np.float64(0.7457098039215686)}

In [30]:
best_layer=best_paras['layer']

View Results on Training set and Test set

In [31]:
discriminator=PCADiscriminator(best_paras['n_components'],embedding_val[:,best_layer,:])
best_split=discriminator.get_best_split(hall_label_val)

In [32]:
discriminator=PCADiscriminator(best_paras['n_components'],embedding_train[:,best_layer,:])
scores=discriminator.get_score()
y_train=(scores>best_split)
print(f'train set acc: {accuracy_score(hall_label_wild,y_train)} AUROC:{roc_auc_score(hall_label_wild,scores)}')

train set acc: 0.8325714285714285 AUROC:0.6960425937698665


In [33]:
discriminator=PCADiscriminator(best_paras['n_components'],embedding_test[:,best_layer,:])
scores=discriminator.get_score()
preds=(scores>best_split)
print(f'test set acc: {accuracy_score(hall_label_test,preds)} AUROC:{roc_auc_score(hall_label_test,scores)}')

test set acc: 0.773 AUROC:0.7018813039251428


Train Linear Prob to Get Better Results

In [34]:
X_train=embedding_train[:, best_layer,:]
y_train=y_train

X_test=embedding_test[:, best_layer,:]
y_test=hall_label_test

In [35]:
best_acc, final_acc, (
            clf, best_state, best_preds, preds, labels_val), losses_train = get_linear_acc(
            X_train,
            y_train,
           X_train,
            y_train,
            2, epochs=50,
            print_ret=True,
            batch_size=512,
            cosine=True,
            nonlinear=True,
            learning_rate=0.05,
            weight_decay=0.0003)

In [ ]:
clf.eval()
output = clf(torch.from_numpy(X_test).to(torch.float32).cuda())
scores_test = torch.sigmoid(output).cpu().data.numpy()
pred_test=scores_test>0.5
print(f'test set acc: {accuracy_score(y_test,pred_test)} AUROC:{roc_auc_score(y_test,scores_test)}')

test set acc: 0.8155 AUROC:0.6313988637744563


: 